In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, when, round
from pyspark.sql.types import DoubleType

In [3]:
path_dat="/content/drive/MyDrive/data/yahoo_aapl_merged.csv"

In [4]:
spark = SparkSession.builder.appName("time data").getOrCreate()



In [5]:
df = spark.read.option("header", "true").csv(path_dat)

In [6]:
df

DataFrame[Date: string, Open: string, High: string, Low: string, Close: string, Adj Close: string, Volume: string, IsDividend: string, DividendAmount: string, IsSplit: string, SplitRatio: string]

In [7]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- IsDividend: string (nullable = true)
 |-- DividendAmount: string (nullable = true)
 |-- IsSplit: string (nullable = true)
 |-- SplitRatio: string (nullable = true)



In [8]:
df = df.withColumn("Date", to_date(col("Date"), "MMM dd, yyyy"))

In [9]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- IsDividend: string (nullable = true)
 |-- DividendAmount: string (nullable = true)
 |-- IsSplit: string (nullable = true)
 |-- SplitRatio: string (nullable = true)



In [10]:
numeric_cols = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
for col_name in numeric_cols:
    df = df.withColumn(col_name, col(col_name).cast(DoubleType()))

In [11]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- IsDividend: string (nullable = true)
 |-- DividendAmount: string (nullable = true)
 |-- IsSplit: string (nullable = true)
 |-- SplitRatio: string (nullable = true)



In [17]:
from pyspark.sql.functions import col, when, sum as spark_sum

In [25]:
df.show()

+----------+----+----+----+-----+---------+-----------+----------+--------------+-------+----------+
|      Date|Open|High| Low|Close|Adj Close|     Volume|IsDividend|DividendAmount|IsSplit|SplitRatio|
+----------+----+----+----+-----+---------+-----------+----------+--------------+-------+----------+
|2007-01-25|3.11|3.16|3.07| 3.08|      2.6| 9.059736E8|     False|           nan|  False|       nan|
|2007-01-26|3.11|3.12|3.04| 3.05|     2.57|  9.86874E8|     False|           nan|  False|       nan|
|2007-01-29|3.08|3.09|3.05| 3.07|     2.59| 9.016644E8|     False|           nan|  False|       nan|
|2007-01-30|3.09|3.09|3.04| 3.06|     2.57| 5.779704E8|     False|           nan|  False|       nan|
|2007-01-31|3.03|3.07|3.01| 3.06|     2.58| 8.560692E8|     False|           nan|  False|       nan|
|2007-02-01|3.08|3.08|3.03| 3.03|     2.55|  6.64342E8|     False|           nan|  False|       nan|
|2007-02-02| 3.0|3.04|2.99| 3.03|     2.55|   6.2153E8|     False|           nan|  False|  

In [26]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

In [28]:
@pandas_udf(StringType())
def clean_split_ratio(s) :
    return s.replace(["nan", None], "0")

In [29]:
df_clean = df.withColumn("SplitRatio", clean_split_ratio(df["SplitRatio"]))

In [31]:
df_clean_ = df_clean.withColumn("DividendAmount", clean_split_ratio(df["DividendAmount"]))

In [32]:
df_clean_.show()

+----------+----+----+----+-----+---------+-----------+----------+--------------+-------+----------+
|      Date|Open|High| Low|Close|Adj Close|     Volume|IsDividend|DividendAmount|IsSplit|SplitRatio|
+----------+----+----+----+-----+---------+-----------+----------+--------------+-------+----------+
|2007-01-25|3.11|3.16|3.07| 3.08|      2.6| 9.059736E8|     False|             0|  False|         0|
|2007-01-26|3.11|3.12|3.04| 3.05|     2.57|  9.86874E8|     False|             0|  False|         0|
|2007-01-29|3.08|3.09|3.05| 3.07|     2.59| 9.016644E8|     False|             0|  False|         0|
|2007-01-30|3.09|3.09|3.04| 3.06|     2.57| 5.779704E8|     False|             0|  False|         0|
|2007-01-31|3.03|3.07|3.01| 3.06|     2.58| 8.560692E8|     False|             0|  False|         0|
|2007-02-01|3.08|3.08|3.03| 3.03|     2.55|  6.64342E8|     False|             0|  False|         0|
|2007-02-02| 3.0|3.04|2.99| 3.03|     2.55|   6.2153E8|     False|             0|  False|  

In [54]:
tata = df_clean_.toPandas()

In [49]:
tata

,Date,Open,High,Low,Close,Adj Close,Volume,IsDividend,DividendAmount,IsSplit,SplitRatio
0,2007-01-25,3.11,3.16,3.07,3.08,2.60,905973600.0,False,0,False,0
1,2007-01-26,3.11,3.12,3.04,3.05,2.57,986874000.0,False,0,False,0
2,2007-01-29,3.08,3.09,3.05,3.07,2.59,901664400.0,False,0,False,0
3,2007-01-30,3.09,3.09,3.04,3.06,2.57,577970400.0,False,0,False,0
4,2007-01-31,3.03,3.07,3.01,3.06,2.58,856069200.0,False,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...
4546,2025-02-19,244.66,246.01,243.16,244.87,244.87,32204200.0,False,0,False,0
4547,2025-02-20,244.94,246.78,244.29,245.83,245.83,32316900.0,False,0,False,0
4548,2025-02-21,245.95,248.69,245.22,245.55,245.55,53197400.0,False,0,False,0
4549,2025-02-24,244.93,248.86,244.42,247.10,247.10,51326400.0,False,0,False,0


In [36]:
import plotly.express as px

fig = px.line(tata, x='Date', y='Adj Close', title='Stock Price Over Time')
fig.show()

In [39]:
fig = px.line(tata, x="Date", y=["Open", "High", "Low", "Close", "Adj Close"],
              title="Stock Price Over Time - Open, High, Low, Close, Adj Close")

In [40]:
fig.show()

In [41]:
tata.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'IsDividend', 'DividendAmount', 'IsSplit', 'SplitRatio'],
      dtype='object')

In [51]:
import pandas as pd

cols_to_plot = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'DividendAmount', 'SplitRatio']
for col in cols_to_plot:
    tata[col] = pd.to_numeric(tata[col])  #


fig = px.line(tata,
              x="Date",
              y=cols_to_plot,
              title="Stock Data Over Time")

fig.show()


In [53]:
fig.show()

In [52]:
tata

,Date,Open,High,Low,Close,Adj Close,Volume,IsDividend,DividendAmount,IsSplit,SplitRatio
0,2007-01-25,3.11,3.16,3.07,3.08,2.60,905973600.0,False,0.0,False,0.0
1,2007-01-26,3.11,3.12,3.04,3.05,2.57,986874000.0,False,0.0,False,0.0
2,2007-01-29,3.08,3.09,3.05,3.07,2.59,901664400.0,False,0.0,False,0.0
3,2007-01-30,3.09,3.09,3.04,3.06,2.57,577970400.0,False,0.0,False,0.0
4,2007-01-31,3.03,3.07,3.01,3.06,2.58,856069200.0,False,0.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4546,2025-02-19,244.66,246.01,243.16,244.87,244.87,32204200.0,False,0.0,False,0.0
4547,2025-02-20,244.94,246.78,244.29,245.83,245.83,32316900.0,False,0.0,False,0.0
4548,2025-02-21,245.95,248.69,245.22,245.55,245.55,53197400.0,False,0.0,False,0.0
4549,2025-02-24,244.93,248.86,244.42,247.10,247.10,51326400.0,False,0.0,False,0.0


In [56]:
fig = px.line(
    tata,
    x='Date',
    y='Adj Close',
    color='IsSplit',
    title='Stock Price Over Time by Split Status'
)

fig.show()

In [61]:
import plotly.express as px

fig = px.line(
    tata,
    x='Date',
    y='Adj Close',
    color='IsDividend',  # Use 'color' instead of 'hue'
    title='Stock Price Over Time by Dividend Status'
)

fig.show()

In [62]:
tata

,Date,Open,High,Low,Close,Adj Close,Volume,IsDividend,DividendAmount,IsSplit,SplitRatio
0,2007-01-25,3.11,3.16,3.07,3.08,2.60,905973600.0,False,0,False,0
1,2007-01-26,3.11,3.12,3.04,3.05,2.57,986874000.0,False,0,False,0
2,2007-01-29,3.08,3.09,3.05,3.07,2.59,901664400.0,False,0,False,0
3,2007-01-30,3.09,3.09,3.04,3.06,2.57,577970400.0,False,0,False,0
4,2007-01-31,3.03,3.07,3.01,3.06,2.58,856069200.0,False,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...
4546,2025-02-19,244.66,246.01,243.16,244.87,244.87,32204200.0,False,0,False,0
4547,2025-02-20,244.94,246.78,244.29,245.83,245.83,32316900.0,False,0,False,0
4548,2025-02-21,245.95,248.69,245.22,245.55,245.55,53197400.0,False,0,False,0
4549,2025-02-24,244.93,248.86,244.42,247.10,247.10,51326400.0,False,0,False,0


from matplotlib import pyplot as plt
tata['Open'].plot(kind='hist', bins=20, title='Open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
tata['High'].plot(kind='hist', bins=20, title='High')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
tata['Low'].plot(kind='hist', bins=20, title='Low')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
tata['Close'].plot(kind='hist', bins=20, title='Close')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(tata['IsSplit'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(tata, x='Open', y='IsSplit', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(tata['SplitRatio'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(tata, x='Open', y='SplitRatio', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(tata['IsDividend'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(tata, x='High', y='IsDividend', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [68]:

tata = df_clean_.drop(*['Open', 'High', 'Low', 'Close'])

In [70]:
tata.show()

+----------+---------+-----------+----------+--------------+-------+----------+
|      Date|Adj Close|     Volume|IsDividend|DividendAmount|IsSplit|SplitRatio|
+----------+---------+-----------+----------+--------------+-------+----------+
|2007-01-25|      2.6| 9.059736E8|     False|             0|  False|         0|
|2007-01-26|     2.57|  9.86874E8|     False|             0|  False|         0|
|2007-01-29|     2.59| 9.016644E8|     False|             0|  False|         0|
|2007-01-30|     2.57| 5.779704E8|     False|             0|  False|         0|
|2007-01-31|     2.58| 8.560692E8|     False|             0|  False|         0|
|2007-02-01|     2.55|  6.64342E8|     False|             0|  False|         0|
|2007-02-02|     2.55|   6.2153E8|     False|             0|  False|         0|
|2007-02-05|     2.53| 5.788524E8|     False|             0|  False|         0|
|2007-02-06|     2.53| 8.643936E8|     False|             0|  False|         0|
|2007-02-07|     2.59|1.0668252E9|     F

In [72]:
tata.write.csv("tata_cleaned", header=True, mode="overwrite")